In [1]:
import pandas as pd
df = pd.read_csv("data/Text/002(done).csv")


In [2]:
df

,1,words,start_time,end_time
0,2,I'm,2.55,NaN
1,3,an,2.73,NaN
2,4,immigrant,2.82,NaN
3,5,I,3.66,NaN
4,6,came,3.80,NaN
...,...,...,...,...
809,811,there,477.90,479.2
810,812,with,479.40,479.6
811,813,my,479.60,479.7
812,814,twin,479.70,480


確定所有的數值是int or float

In [3]:
df['start_time'] = pd.to_numeric(df['start_time'])
df['end_time'] = pd.to_numeric(df['end_time'])


ValueError: Unable to parse string " " at position 804

抓出最後一個字的時間

In [ ]:
last_word_time = df.tail(1).values[0,2]

將每十秒的timeframe的字組成list

In [ ]:
all_sentences001 = []
for t in range(0,480):
    sentences = []
    for index, rows in df.loc[(df["start_time"]>=t) & (df["start_time"]<=(t+10))].iterrows():
        sentences.append(rows["words"])
    all_sentences001.append(" ".join(sentences))

In [ ]:
for i in range(1):
    print("=",all_sentences001[i],"=")

= hello my name is Angelinis and  I am 21 years old I was born in Bogota Colombia and in the place I used =


## the SBERT

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
model = SentenceTransformer('all-MiniLM-L6-v2')
SBERT = []

def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
# Two lists of sentences
for i in range(0,len(all_sentences001)-1):
    #Compute embedding for both lists
    embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
    embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    # convert from PyTorch tensor to numpy array
    #print(get_cosine_similarity(embeddings1, embeddings2))
    SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
######


0.9482838
0.8144995
0.98601556
0.88233733
0.9117321
0.87547493
0.8069012
0.64813983
0.9320267
0.79163826
0.8841871
0.9513539
0.8379276
0.9398347
0.92691046
0.9870994
0.9031306
0.8240187
0.8508369
0.78724015
0.9551784
0.78759396
0.84378266
0.8690597
0.97452855
0.97081476
0.97538686
0.8606863
0.89825696
0.97958064
0.9803824
0.8489533
0.86509454
0.54132223
0.8190703
0.8658678
0.96830803
0.7903678
0.8629867
0.7121433
0.9865939
0.8236573
0.95243585
0.9116342
0.62665
0.94617176
0.9724674
0.8434484
1.0
0.7297085
0.84631133
0.89877886
0.7921372
0.9976187
0.9630146
0.90134525
0.93958646
0.8902681
0.992371
0.9968934
0.89163065
0.9154493
0.93578
0.9712582
0.95807046
0.9027128
0.8993736
0.74630284
0.9080129
0.8805833
0.9622211
0.8737424
0.9089768
0.8134278
0.9331467
1.0
0.96080506
0.97757107
0.72362375
0.8039621
0.9657691
0.96637285
0.9306378
0.7910426
0.9795533
0.9357407
0.9412188
0.744015
0.9295281
0.8322691
0.9335833
0.8506999
0.99999994
0.8593014
0.98947084
0.92923105
0.81612045
0.5166989
0.96

## The Wu-Pulmer Similarity

In [ ]:
S = all_sentences001[0].split()
print(S)

['hello', 'my', 'name', 'is', 'Angelinis', 'and', 'I', 'am', '21', 'years', 'old', 'I', 'was', 'born', 'in', 'Bogota', 'Colombia', 'and', 'in', 'the', 'place', 'I', 'used']


### 製作去掉stopwords 的time frame的文字

In [ ]:
from nltk.corpus import stopwords
all_sentences001 = []
stops = set(stopwords.words('english'))
for t in range(0,480):
    sentences = []
    for index, rows in df.loc[(df["start_time"]>=t) & (df["start_time"]<=(t+10))].iterrows():
        if rows["words"].lower() not in stops:
            sentences.append(rows["words"].lower())
    all_sentences001.append(" ".join(sentences))

## 算出每個timeframe內每兩個字的每兩個synset的wu-pulmer similarity

In [ ]:
import nltk
#nltk.download('omw-1.4')
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(stops)
all_sentences_wu_sum = []
for sentence_num in range(0,len(all_sentences001)): #現在第幾句
    wu_pulmer_sum = 0
    the_sentence = str(all_sentences001[sentence_num]).split()
    #print(the_sentence)
    timeframe_pairs_count = 0
    for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
        if len(wordnet.synsets(the_sentence[words_num])) > 0 :
            syns1 = wordnet.synsets(the_sentence[words_num])
            syns1_num = len(syns1)
        else:
            continue
        for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
            if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                syns2 = wordnet.synsets(the_sentence[words_num2])
                syns2_num = len(syns2)
                for i in range(0, syns1_num):
                    for u in range(0, syns2_num):
                        #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                        wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                        timeframe_pairs_count += 1
            else:
                continue
    if timeframe_pairs_count != 0:
        all_sentences_wu_sum.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
    else: 
        all_sentences_wu_sum.append(wu_pulmer_sum)


{'in', 'between', "doesn't", 're', 'it', 'now', 'through', 'didn', 'did', 'have', 'by', "that'll", 'over', 'itself', 'ma', 'into', "wasn't", 'a', 'own', "needn't", 'if', 'what', 'that', 'just', 've', "haven't", 'same', 'again', 'your', "shan't", 'can', 'does', 'them', 'been', 'needn', 'ours', 'don', 'where', "won't", 'ain', 'ourselves', 'him', 'were', 'out', 'not', "hadn't", 'mustn', 'our', 'having', 'under', 'how', "aren't", 'these', 'too', 'doesn', 'myself', 'she', 'further', 'weren', 'until', 'at', 'yourself', 'who', 'wasn', 'on', 'below', 'because', 'against', 'only', 'hers', 'mightn', 'with', 'very', 'to', 'hasn', 'off', 'about', 'they', 'or', 'had', 'himself', 'theirs', 'will', 'down', 'but', 'of', 'few', 'is', 'yourselves', 'such', 'which', 'has', 'once', 'wouldn', 'won', 'both', "wouldn't", 'their', 'during', 'then', 'll', 'no', 'shan', 'isn', "you've", 'o', 'should', 'an', 'm', "should've", 'whom', "shouldn't", 'when', "you'd", 'its', 'from', 'be', 'y', 'while', "don't", 'her'

In [ ]:
#print(all_sentences_wu_sum)
wu_difference = []
for x in range(len(all_sentences_wu_sum)-1):
    wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])

In [ ]:
print(wu_difference)

[ 2.51598642e-02  6.80760669e-03  0.00000000e+00 -8.87181684e-03
 -4.05529136e-03 -7.75674827e-03 -1.08837018e-02 -1.00857302e-03
  1.35245633e-02  3.08225699e-03 -1.30394156e-02  0.00000000e+00
 -2.82075125e-03  9.96625142e-03 -8.80574949e-03  0.00000000e+00
  5.37586222e-03  4.97605579e-03 -8.28021122e-03  1.96326960e-02
  0.00000000e+00 -2.25711773e-03 -1.57273377e-03 -1.14894716e-02
  2.81534515e-02  1.35762315e-02  0.00000000e+00  1.33205897e-02
 -2.19776246e-02 -1.35266203e-02  0.00000000e+00 -2.24440318e-02
 -7.35788048e-03 -3.09465659e-02  3.88022965e-02 -4.21079553e-02
  0.00000000e+00  4.63199243e-03 -1.92408231e-02  7.03878313e-02
  1.52918284e-02  6.06910102e-02 -8.26518204e-03  0.00000000e+00
  7.99947070e-03 -1.16854111e-02 -6.73787949e-03 -1.44254632e-02
  0.00000000e+00 -2.81487819e-03 -1.19833612e-02 -1.60107506e-02
  1.29539317e-02  0.00000000e+00  0.00000000e+00 -8.92302939e-03
  3.99367205e-04 -1.40638754e-02  0.00000000e+00  4.05289751e-03
 -1.64600706e-03 -7.90410

# Normalization

In [ ]:
from sklearn import preprocessing
import numpy as np
wu_difference = np.array(wu_difference)
SBERT = np.array(SBERT)
#normalized_SBERT = preprocessing.normalize([SBERT])
#normalized_wu_difference = preprocessing.normalize([wu_difference])


min_max_scaler = preprocessing.MinMaxScaler()
SBERT = np.reshape(SBERT,(479,1))
wu_difference = np.reshape(wu_difference,(479,1))
print(SBERT)
normalized_SBERT  = min_max_scaler.fit_transform(SBERT)
normalized_wu_difference  = min_max_scaler.fit_transform(wu_difference)

[[0.9482838 ]
 [0.8144995 ]
 [0.98601556]
 [0.88233733]
 [0.9117321 ]
 [0.87547493]
 [0.8069012 ]
 [0.64813983]
 [0.9320267 ]
 [0.79163826]
 [0.8841871 ]
 [0.9513539 ]
 [0.8379276 ]
 [0.9398347 ]
 [0.92691046]
 [0.9870994 ]
 [0.9031306 ]
 [0.8240187 ]
 [0.8508369 ]
 [0.78724015]
 [0.9551784 ]
 [0.78759396]
 [0.84378266]
 [0.8690597 ]
 [0.97452855]
 [0.97081476]
 [0.97538686]
 [0.8606863 ]
 [0.89825696]
 [0.97958064]
 [0.9803824 ]
 [0.8489533 ]
 [0.86509454]
 [0.54132223]
 [0.8190703 ]
 [0.8658678 ]
 [0.96830803]
 [0.7903678 ]
 [0.8629867 ]
 [0.7121433 ]
 [0.9865939 ]
 [0.8236573 ]
 [0.95243585]
 [0.9116342 ]
 [0.62665   ]
 [0.94617176]
 [0.9724674 ]
 [0.8434484 ]
 [1.        ]
 [0.7297085 ]
 [0.84631133]
 [0.89877886]
 [0.7921372 ]
 [0.9976187 ]
 [0.9630146 ]
 [0.90134525]
 [0.93958646]
 [0.8902681 ]
 [0.992371  ]
 [0.9968934 ]
 [0.89163065]
 [0.9154493 ]
 [0.93578   ]
 [0.9712582 ]
 [0.95807046]
 [0.9027128 ]
 [0.8993736 ]
 [0.74630284]
 [0.9080129 ]
 [0.8805833 ]
 [0.9622211 ]
 [0.87

In [ ]:
for i in range(10):
    print(normalized_SBERT[i],normalized_wu_difference[i])


[0.9178506] [0.83147301]
[0.7053398] [0.78501504]
[0.97778594] [0.76778187]
[0.81309736] [0.74532323]
[0.8597897] [0.75751607]
[0.80219674] [0.74814598]
[0.6932702] [0.74023023]
[0.44108427] [0.76522871]
[0.8920268] [0.80201873]
[0.66902566] [0.77558447]


In [ ]:
print(np.max(normalized_SBERT))

1.0
